In [ ]:
import asmsa
import mdtraj as md
import nglview as nv
import numpy as np

In [ ]:
%pip install DFO-LS

In [ ]:
import dfols

In [ ]:
conf = "trpcage_correct.pdb"
topol = "topol_correct.top"
index = 'index_correct.ndx'

In [ ]:
# outputs of prepare.ipynb

#train_tr = md.load('x_train.xtc',top=conf)
#test_tr = md.load('x_test.xtc',top=conf)

train_tr = md.load('train.xtc',top=conf)
test_tr = md.load('test.xtc',top=conf)

In [ ]:
train_g = np.moveaxis(train_tr.xyz,0,-1)
test_g = np.moveaxis(test_tr.xyz,0,-1)

In [ ]:
train_g.shape

In [ ]:
sparse_dists = asmsa.NBDistancesSparse(train_g.shape[0], density=2)
mol = asmsa.Molecule(pdb=conf,top=topol,ndx=index,fms=[sparse_dists])

In [ ]:
train_int = mol.intcoord(train_g)
train_int.shape

In [ ]:
test_int = mol.intcoord(test_g)
test_int.shape

In [ ]:
# faking ... true output of decoder predicting something should come here

out_idx = 100

dec_out = test_int[:,out_idx]
dec_out.shape

In [ ]:
diff = train_int.T - dec_out
msd = np.sum(diff * diff,axis=1)

In [ ]:
minidx = np.argmin(msd)
x0 = train_g[:,:,minidx].flatten()

In [ ]:
def minf(x,target,mol):
    x_ = np.reshape(x,(-1,3,1))
    int = np.reshape(mol.intcoord(x_),(-1,))
    return int-target

In [ ]:
sol = dfols.solve(minf,x0,args=(dec_out,mol),maxfun=5000)

In [ ]:
solm = md.load_pdb(conf)
solm.xyz = np.reshape(sol.x,(1,-1,3))
#solm.xyz = np.reshape(x0,(1,-1,3))
v = nv.show_mdtraj(solm)

refm = md.load_pdb(conf)
refm.xyz = test_g[:,:,out_idx]

v.add_component(refm,color='red')
v.clear(component=0)
v.clear(component=1)

v.add_representation('ribbon',component=1)
v.add_representation('licorice',component=0)

v

In [ ]:
test_g.shape